# 청와대 청원 NLP Topic 분류 모델

#### Github 에서 데이터 로드 및 확인

In [52]:
!mkdir data
!wget -O /content/data/train.csv https://raw.githubusercontent.com/Adn-thkim/topic_classification_bluehouse_petition/master/data/train.csv

mkdir: cannot create directory ‘data’: File exists
--2022-01-24 10:59:20--  https://raw.githubusercontent.com/Adn-thkim/topic_classification_bluehouse_petition/master/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53439079 (51M) [text/plain]
Saving to: ‘/content/data/train.csv’

/content/data/train 100%[===================>]  50.96M   192MB/s    in 0.3s    

2022-01-24 10:59:21 (192 MB/s) - ‘/content/data/train.csv’ saved [53439079/53439079]



In [72]:
import pandas as pd

bh_train = pd.read_csv('/content/data/train.csv', encoding = 'utf-8')
bh_train.head()

,index,category,data
0,0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...
1,1,0,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,2,1,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올..."
3,3,1,"티비 12세,15세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠..."
4,4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...


# 데이터 전처리

#### 데이터 형태 확인
- Index, category, data 세가지 속성으로 이뤄진 text 데이터 40k 개
- Index 속성을 불필요하므로 제거

In [76]:
print(bh_train.columns)
print(bh_train.shape)

Index(['category', 'data'], dtype='object')
(40000, 2)


In [74]:
bh_train = bh_train.iloc[:, 1:]
print(bh_train.columns)
print(bh_train.shape)

Index(['category', 'data'], dtype='object')


#### 중복 데이터 확인
- 중복 데이터가 615개 있는 것을 확인하고 중복된 데이터 출력
- 중복데이터 제거

In [80]:
duplicated = bh_train.duplicated(subset = ['category', 'data'])
print(duplicated.sum())

615


In [88]:
bh_train.iloc[duplicated.to_list(),:]

,category,data
402,2,청소년보호법폐지. 청소년보호법폐지
920,1,영화 5천원 마지막 수요일에서 매주 수요일로 확대해야. 영화 5천원 마지막 수요일에...
1270,1,영화 5천원 마지막 수요일에서 매주 수요일로 확대해야. 영화 5천원 마지막 수요일에...
1812,0,조두순 출소 반대. 조두순 출소 반대합니다.
2140,1,축구장에 있는 천연 잔디좀 신경 써주세요!!. 지금 축구장에 천연 잔디 모습이 안 ...
...,...,...
39856,2,소년법 개정과 부산 폭행사건 가해자 엄중 처벌을 청원합니다.. 소년법 개정\n어리다...
39867,0,청소년보호법 폐지해 주세요!!. 인천8세여아살인사건과 부산여중생폭행사건 너무 잔인합...
39893,2,2020년도 대학 입시. 현재 고2 자녀를 둔 학부모 입니다\n오늘 뉴스를 보니 ...
39895,1,2018월드컵. 안녕 하세요 저는 중학교2학년인 학생 입 니다. 저는 신 태용 감...


In [75]:
clean_data = bh_train.drop_duplicates(subset = ['category' ,'data'], inplace = False)

In [89]:
print(clean_data.shape)
print(clean_data.head())

(39385, 2)
   category                                               data
0         2  신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...
1         0  학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2         1  빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올...
3         1  티비 12세,15세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠...
4         1  무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...
<class 'pandas.core.frame.DataFrame'>
